In [ ]:
import cv2
cap=cv2.VideoCapture(0)

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    image=Image.open(image_path)
    # the array based representation of the image will be used later to prepare the 
    #result image with boxes and labels on it.
    image_np=load_image_into_numpy_array(image)

In [ ]:
while True:
    ret,image_np=cap.read()

In [ ]:
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)
plt.show()

In [ ]:
cv2.imshow('object detection',cv2.resize(image_np,(800,600)))
if cv2.destroyAllWindows()
break

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
cap=cv2.VideoCapture(1)
#This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
# ## Object detection imports
# Here are the imports from the object detection module.

In [ ]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [ ]:
# # Model preparation
# ## Variables
#
#Any model exported using the 'export_interference_graph.py' tool can be loaded here simply by changing 'PATH_TO_CKPT' to point to a new .pb file.
#
#By default we use an "SSD with Mobilenet" model here.See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
#what model to download.
MODEL_NAME='ssd_moilenet_v1_coco_11_06_2017'
MODEL_FILE=MODEL_NAME+'.tar.gz'
DOWNLOAD_BASE='https://github.com/tensorflow/models/tree/master/research/object_detection/'
#Path to frozen detection graph.This is actual model that is used for object detection.
PATH_TO_CKPT=MODEL_NAME+'/frozen_inference_graph.pb'
#List of the strings that is used to correct label for each box.
PATH_TO_LABELS=os.path.join('data','mscoco_label_map.pbtxt')
NUM_CLASSES=90

# ## Download Model

In [ ]:
opener=urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE+MODEL_FILE,MODEL_FILE)
tar_file=tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name=os.path.basename(file_name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file,os.getcwd())
        
        
 # ## Load a (frozen) Tensorflow model into memory.       

In [ ]:
detection_graph=tf.Graph()
with detection.as_default():
    od_graph.as_default=tf.GraphDef()
    with tf.gfiloe.GFile(PATH_TO_CKPT,'rb') as fid:
        serialized_graph=fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def,name='')
# ##Loading label map
#Label maps map indices to category names,so that when convolution network predicts'5',we know that this corresponds to 'airplane'.Here we use internal utility function,but anything that return a dictionary mapping integers

In [ ]:
label_map=label_map_util.load_labelmap(PATH_TO_LABELS)
categories=label_map_util.converts_label_map_to_catagories(label_map,max_num_classes=NUM_CLASSES,use_display_name=True)
category_index=label_map_util.create_category_index(categories)

# ##Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width,im_height)=image.size
    return np.array(image.getdata()).reshape(
    (im_height,im_width,3)).astype(np.uint8)
# # Detection
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images,just add path to the imagesto the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGE_DIR='test_images'
TEST_IMAGE_PATHS=[os.path.join(PATH_TO_TEST_IMAGE_DIR,'images.jpg'.format(i)) for i in range(1,3)]

#Size,in inchies,of the output images.
IMAGE_SIZE=(12,8)

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            ret,image_np=cap.read()
            #Expand dimensions since the model images to have shape:[1,None,None,3]
            image_np_expanded=np.expand_dims(image_np,axis=0)
            image_tensor=detection_graph.get_tensor_by_name('image_tensor:0')
            #Each box represents a part of the image where a particular object was detected.
            boxes=detection_graph.get_tensor_by_name('detection_scores:0')
             # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

            cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
            break